# LLM Distillation: Offline Distillation

<div class="align-center">

<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a>
</div>

## วัตถุประสงค์การเรียนรู้


1. เข้าใจว่า **offline distillation** คืออะไรและแตกต่างจาก fine-tuning ปกติอย่างไร
2. สร้างชุดข้อมูลสำหรับ training จาก **teacher model**
3. ฝึก **student model** ขนาดเล็กกว่าให้เลียนแบบ output ของ teacher
4. ใช้ **Unsloth + QLoRA** สำหรับการ training student อย่างมีประสิทธิภาพ
5. เปรียบเทียบประสิทธิภาพและความเร็วระหว่าง student กับ teacher
6. ศึกษาโมเดล distilled ระดับ production (DeepSeek-R1-Distill)


## Knowledge Distillation คืออะไร?

**Knowledge Distillation** เป็นการถ่ายโอนความรู้จาก **teacher model** ขนาดใหญ่ไปยัง **student model** ขนาดเล็กกว่า

### กระบวนการ Offline Distillation:

```
1. Teacher Model → สร้าง responses สำหรับข้อมูล training
2. บันทึก teacher outputs (offline dataset)
3. Student Model → เรียนรู้เพื่อเลียนแบบ responses ของ teacher
4. Deploy student model ที่เล็กกว่าและเร็วกว่า
```

## ส่วนที่ 1: การติดตั้ง

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Colab-specific installation
    import torch; v = re.match(r"[0-9\\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
# ตรวจสอบ GPU
!nvidia-smi

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## ส่วนที่ 2: โหลดชุดข้อมูล Instruction

เราจะใช้ชุดข้อมูล **Alpaca** ที่มีตัวอย่าง instruction-following

In [ ]:
print("="*60)
print("กำลังโหลดชุดข้อมูล ALPACA")
print("="*60)

# โหลดชุดข้อมูล Alpaca
dataset = load_dataset("yahma/alpaca-cleaned", split="train")

# ใช้ชุดข้อมูลย่อยเพื่อความรวดเร็ว (300 ตัวอย่าง)
dataset = dataset.shuffle(seed=42).select(range(300))

print(f"\nขนาดชุดข้อมูล: {len(dataset):,} ตัวอย่าง")
print("\nตัวอย่างจากชุดข้อมูล:")
print("="*60)
print(f"Instruction: {dataset[0]['instruction']}")
print(f"Input: {dataset[0]['input']}")
print(f"Output: {dataset[0]['output']}")
print("="*60)

In [ ]:
def format_instruction(example):
    """จัดรูปแบบชุดข้อมูลเป็น instruction prompts"""
    if example["input"].strip():
        return f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
"""
    else:
        return f"""### Instruction:
{example['instruction']}

### Response:
"""

# แสดงตัวอย่าง formatted prompt
print("ตัวอย่าง formatted prompt:")
print("="*60)
print(format_instruction(dataset[0]))
print(f"Output ที่คาดหวัง: {dataset[0]['output']}")
print("="*60)

## ส่วนที่ 3: โหลด Teacher Model

เราจะใช้ **Gemma-2-2B-Instruct** เป็น teacher model -

ในการใช้งานจริง teachers มักจะใหญ่กว่ามาก (7B, 70B, 405B)

In [ ]:
print("\n" + "="*60)
print("กำลังโหลด TEACHER MODEL: Gemma-2-2B-Instruct")
print("="*60)

teacher_model_name = "unsloth/gemma-2-2b-it"

print(f"กำลังโหลด teacher model: {teacher_model_name}")
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)
teacher_model = AutoModelForCausalLM.from_pretrained(
    teacher_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)

print(f"\n✓ โหลด teacher model เรียบร้อย: Gemma-2-2B (2.5B พารามิเตอร์)")
print(f"  Model dtype: {teacher_model.dtype}")

## ส่วนที่ 4: สร้าง Teacher Outputs (Offline Distillation Dataset)

นี่คือส่วน **"offline"** - เราสร้าง teacher responses ครั้งเดียวแล้วบันทึก


In [ ]:
# สร้าง teacher pipeline
teacher_pipe = pipeline(
    "text-generation",
    model=teacher_model,
    tokenizer=teacher_tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# ทดสอบกับตัวอย่างหนึ่ง
test_prompt = format_instruction(dataset[0])
test_output = teacher_pipe(test_prompt)[0]['generated_text']

print("Teacher test output:")
print("="*60)
print(test_output)
print("="*60)

In [ ]:
print("\n" + "="*60)
print("กำลังสร้าง TEACHER OUTPUTS สำหรับ DISTILLATION")
print("="*60)
teacher_outputs = []
batch_size = 8

for i in tqdm(range(0, len(dataset), batch_size), desc="กำลังสร้าง teacher outputs"):
    batch_indices = range(i, min(i+batch_size, len(dataset)))
    prompts = [format_instruction(dataset[idx]) for idx in batch_indices]

    # สร้าง teacher responses
    outputs = teacher_pipe(prompts, batch_size=batch_size)

    for prompt, output in zip(prompts, outputs):
        generated_text = output[0]['generated_text']
        # ดึงเฉพาะส่วน response (หลังจาก prompt)
        response = generated_text[len(prompt):].strip()
        teacher_outputs.append({
            'prompt': prompt,
            'teacher_response': response
        })

print(f"\n✓ สร้าง teacher outputs แล้ว {len(teacher_outputs):,} ตัวอย่าง")
print("\nตัวอย่าง teacher output:")
print("="*60)
print(f"Prompt: {teacher_outputs[0]['prompt'][:100]}...")
print(f"Teacher Response: {teacher_outputs[0]['teacher_response']}")
print("="*60)

In [ ]:
# บันทึก teacher outputs สำหรับใช้ในอนาคต
with open('teacher_outputs_distillation.json', 'w') as f:
    json.dump(teacher_outputs, f, indent=2)

print("✓ บันทึก teacher outputs ไปที่ 'teacher_outputs_distillation.json'")

# ปล่อยหน่วยความจำของ teacher model
del teacher_model
del teacher_pipe
torch.cuda.empty_cache()

print("✓ ถอด teacher model เพื่อปล่อยหน่วยความจำ GPU")

## ส่วนที่ 5: เตรียมชุดข้อมูล Student Training

แปลง teacher outputs เป็นรูปแบบ training สำหรับ student model

In [ ]:
from unsloth.chat_templates import get_chat_template

print("\n" + "="*60)
print("กำลังเตรียมชุดข้อมูล DISTILLATION")
print("="*60)

# สร้าง training texts: prompt + teacher response
training_data = []
for item in teacher_outputs:
    # Student เรียนรู้เพื่อสร้าง response ของ teacher จาก prompt
    full_text = item['prompt'] + item['teacher_response']
    training_data.append({'text': full_text})

distillation_dataset = Dataset.from_list(training_data)

# แบ่งเป็น train/test
train_size = int(0.9 * len(distillation_dataset))
train_dataset = distillation_dataset.select(range(train_size))
test_dataset = distillation_dataset.select(range(train_size, len(distillation_dataset)))

print(f"\n✓ สร้างชุดข้อมูล distillation เรียบร้อย")
print(f"  ตัวอย่าง training: {len(train_dataset):,}")
print(f"  ตัวอย่าง test: {len(test_dataset):,}")
print("\nแสดงตัวอย่าง training:")
print("="*60)
print(train_dataset[0]['text'][:500] + "...")
print("="*60)

## ส่วนที่ 6: โหลด Student Model ด้วย Unsloth

เราจะใช้ **Qwen2.5-0.5B** เป็น student - เล็กกว่า teacher 5 เท่า!

- **Teacher**: Gemma-2-2B (2.5B พารามิเตอร์)
- **Student**: Qwen2.5-0.5B (0.5B พารามิเตอร์)

In [ ]:
from unsloth import FastModel

print("\n" + "="*60)
print("กำลังโหลด STUDENT MODEL: Qwen2.5-0.5B")
print("="*60)

# ล้างหน่วยความจำ GPU
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

student_model, student_tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-0.5B",
    max_seq_length = 2048,
    load_in_4bit = True,  # ใช้ 4-bit quantization เพื่อประสิทธิภาพ
    dtype = None,  # Auto-detect
)

base_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"\n✓ โหลด student model เรียบร้อย: Qwen2.5-0.5B (0.5B พารามิเตอร์)")
print(f"  หน่วยความจำ base model: {base_memory} GB")
print(f"  Model dtype: {student_model.dtype}")
print(f"\n  การเปรียบเทียบขนาด: Student เล็กกว่า ~5 เท่า!")

## ส่วนที่ 7: เพิ่ม LoRA Adapters ให้ Student

เราจะใช้ QLoRA เพื่อฝึก student model อย่างมีประสิทธิภาพ

In [ ]:
print("\n" + "="*60)
print("กำลังเพิ่ม LoRA ADAPTERS ให้ STUDENT")
print("="*60)

student_model = FastModel.get_peft_model(
    student_model,
    r = 16,  # LoRA rank
    lora_alpha = 16,  # LoRA scaling
    lora_dropout = 0,  # Unsloth optimized

    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],

    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# นับพารามิเตอร์ที่ฝึกได้
trainable_params = sum(p.numel() for p in student_model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in student_model.parameters())
trainable_percent = 100 * trainable_params / all_params

print(f"\n✓ เพิ่ม LoRA adapters เรียบร้อย")
print(f"  พารามิเตอร์ที่ฝึกได้: {trainable_params:,}")
print(f"  พารามิเตอร์ทั้งหมด: {all_params:,}")
print(f"  เปอร์เซ็นต์ที่ฝึกได้: {trainable_percent:.2f}%")

lora_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"\n  หน่วยความจำพร้อม LoRA: {lora_memory} GB")
print(f"  หน่วยความจำเพิ่มเติมสำหรับ LoRA: {lora_memory - base_memory:.3f} GB")

## ส่วนที่ 8: กำหนดค่าการ Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

print("\n" + "="*60)
print("กำลังกำหนดค่าการ TRAINING")
print("="*60)

trainer = SFTTrainer(
    model = student_model,
    tokenizer = student_tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = DataCollatorForSeq2Seq(tokenizer = student_tokenizer),
    dataset_num_proc = 2,

    args = TrainingArguments(
        output_dir = "distilled_student_outputs",

        # ระยะเวลา training (ปรับแต่งสำหรับ 4-5 นาที)
        num_train_epochs = 1,
        max_steps = -1,

        # การตั้งค่า batch
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,  # Effective batch size = 8

        # Learning rate
        learning_rate = 2e-4,
        warmup_steps = 10,

        # Optimization
        optim = "adamw_8bit",
        weight_decay = 0.01,

        # Precision
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),

        # Logging และ saving
        logging_steps = 10,
        eval_strategy = "steps",
        eval_steps = 50,
        save_strategy = "epoch",
        save_total_limit = 1,

        # Misc
        seed = 3407,
        report_to = "none",
    ),
)

print("\n กำหนดค่า training สำหรับ distillation อย่างรวดเร็ว")
print(f"  ตัวอย่าง training: {len(train_dataset):,}")
print(f"  Effective batch size: 8")
print(f"  เวลา training โดยประมาณ: 4-5 นาที")

## ส่วนที่ 9: ฝึก Student Model (Distillation)

ถึงเวลาที่จะ distill ความรู้ของ teacher เข้าไปใน student!

In [ ]:
import time

print("\n" + "="*60)
print("เริ่มการ DISTILLATION TRAINING")
print("="*60)
print("Student กำลังเรียนรู้เพื่อเลียนแบบ responses ของ teacher...\n")

torch.cuda.reset_peak_memory_stats()
start_time = time.time()

# Train!
trainer_stats = trainer.train()

training_time = time.time() - start_time
peak_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)

print("\n" + "="*60)
print("การ DISTILLATION TRAINING เสร็จสมบูรณ์!")
print("="*60)
print(f"เวลา training: {training_time/60:.2f} นาที")
print(f"หน่วยความจำ GPU สูงสุด: {peak_memory} GB")
print(f"Training loss สุดท้าย: {trainer_stats.training_loss:.4f}")
print("="*60)

## ส่วนที่ 10: ทดสอบ Distilled Student Model

มาเปรียบเทียบ distilled student กับตัวอย่างชุดข้อมูลเดิม

In [ ]:
def test_student(model, tokenizer, prompt, max_tokens=200):
    """สร้าง response จาก student model"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens = max_tokens,
        temperature = 0.7,
        top_p = 0.9,
        do_sample = True,
        pad_token_id = tokenizer.eos_token_id,
    )

    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response

# เลือกตัวอย่างทดสอบ
test_indices = [10, 25, 50]

print("\n" + "="*60)
print("กำลังทดสอบ DISTILLED STUDENT MODEL")
print("="*60)

for idx in test_indices:
    example = dataset[idx]
    prompt = format_instruction(example)

    print(f"\n[ตัวอย่างที่ {idx}]")
    print("-"*60)
    print(f"Instruction: {example['instruction']}")
    if example['input'].strip():
        print(f"Input: {example['input']}")
    print("-"*60)

    # สร้าง student response
    student_response = test_student(student_model, student_tokenizer, prompt)

    print(f"Student Response (Distilled):\n{student_response}")
    print("-"*60)
    print(f"Teacher Response (จาก Dataset):\n{teacher_outputs[idx]['teacher_response']}")
    print("-"*60)
    print(f"Ground Truth (ต้นฉบับ):\n{example['output']}")
    print("="*60)

## ส่วนที่ 11: เปรียบเทียบ Student vs Teacher

มาเปรียบเทียบขนาดโมเดล ความเร็ว และการใช้หน่วยความจำ

In [ ]:
print("\n" + "="*60)
print("การเปรียบเทียบโมเดล: TEACHER vs STUDENT")
print("="*60)

comparison_data = {
    'ตัวชี้วัด': [
        'โมเดล',
        'พารามิเตอร์',
        'อัตราส่วนขนาด',
        'เวลา Training',
        'การใช้หน่วยความจำ',
        'ความเร็ว Inference (โดยประมาณ)',
        'กรณีการใช้งาน'
    ],
    'Teacher (Gemma-2-2B)': [
        'Gemma-2-2B-Instruct',
        '2.5B',
        '1x (baseline)',
        'N/A (pre-trained)',
        '~5-6 GB',
        '1x (baseline)',
        'สร้างข้อมูล training'
    ],
    'Student (Qwen2.5-0.5B)': [
        'Qwen2.5-0.5B + LoRA',
        '0.5B',
        'เล็กกว่า 5 เท่า',
        f'{training_time/60:.1f} นาที',
        f'{peak_memory} GB',
        '~เร็วกว่า 3-5 เท่า',
        'การ deploy ในระบบจริง'
    ]
}

df = pd.DataFrame(comparison_data)
print("\n", df.to_string(index=False))
print("\n" + "="*60)

print("\nข้อดีหลักของ Distilled Student:")
print("  ✓ ขนาดโมเดลเล็กกว่า 5 เท่า")
print("  ✓ Inference เร็วกว่า 3-5 เท่า")
print("  ✓ ใช้หน่วยความจำน้อยกว่า")
print("  ✓ เรียนรู้ความรู้ของ teacher")
print("  ✓ เหมาะสำหรับ edge/mobile deployment")

## ส่วนที่ 12: บันทึก Distilled Model

In [ ]:
print("\n" + "="*60)
print("กำลังบันทึก DISTILLED STUDENT MODEL")
print("="*60)

# บันทึกเฉพาะ LoRA adapters (ขนาดเล็ก)
student_model.save_pretrained("distilled_student_lora")
student_tokenizer.save_pretrained("distilled_student_lora")
print("\n✓ บันทึก LoRA adapters ไปที่: distilled_student_lora/")

# ตรวจสอบขนาด adapter
import os
adapter_path = "distilled_student_lora/adapter_model.safetensors"
if os.path.exists(adapter_path):
    size_mb = os.path.getsize(adapter_path) / (1024 * 1024)
    print(f"  ขนาด adapter: {size_mb:.1f} MB")

# บันทึก merged model (เสริม)
student_model.save_pretrained_merged(
    "distilled_student_merged",
    student_tokenizer,
    save_method = "merged_16bit",
)
print("\n✓ บันทึก merged model ไปที่: distilled_student_merged/")

## ส่วนที่ 13: ทดลองโมเดล Distilled ระดับ Production

มาทดสอบ **DeepSeek-R1-Distill-Qwen-1.5B** - โมเดล distilled ระดับ production

In [ ]:
# ล้าง student model
del student_model
del trainer
torch.cuda.empty_cache()

print("\n" + "="*60)
print("กำลังโหลดโมเดล DISTILLED ระดับ PRODUCTION")
print("DeepSeek-R1-Distill-Qwen-1.5B")
print("="*60)

deepseek_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

deepseek_tokenizer = AutoTokenizer.from_pretrained(deepseek_model_name)
deepseek_model = AutoModelForCausalLM.from_pretrained(
    deepseek_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"\n✓ โหลดโมเดลแล้ว: {deepseek_model_name}")
print(f"  พารามิเตอร์: ~1.5B")
print("\nคุณสมบัติหลัก:")
print("  - Distilled จาก DeepSeek-R1 (reasoning model ที่ใหญ่กว่ามาก)")
print("  - รักษาความสามารถด้าน reasoning ที่แข็งแกร่ง")

In [ ]:
# สร้าง pipeline
deepseek_pipe = pipeline(
    "text-generation",
    model=deepseek_model,
    tokenizer=deepseek_tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

print(" DeepSeek pipeline พร้อม")

### ทดสอบ DeepSeek กับงาน Reasoning

In [ ]:
# Test with reasoning tasks
reasoning_tests = [
    {
        "name": "Math Reasoning",
        "prompt": """Solve this problem step by step:

A store sells notebooks for $3 each and pens for $2 each. If you buy 5 notebooks and 8 pens, how much will you pay in total?

Answer:"""
    },
    {
        "name": "Logic Puzzle",
        "prompt": """Answer this logic puzzle:

If all cats are animals, and some animals are pets, can we conclude that all cats are pets?

Explain your reasoning:"""
    },
    {
        "name": "Code Generation",
        "prompt": """Write a Python function to check if a string is a palindrome.

Function:"""
    }
]

print("\n" + "="*60)
print("กำลังทดสอบโมเดล DISTILLED ระดับ PRODUCTION (DeepSeek)")
print("="*60)

for test in reasoning_tests:
    print(f"\n[{test['name']}]")
    print("-"*60)
    print(test['prompt'])
    print("\nDeepSeek Response:")
    print("-"*60)

    response = deepseek_pipe(test['prompt'], max_new_tokens=300)[0]['generated_text']
    print(response[len(test['prompt']):])
    print("="*60)

## ส่วนที่ 14: การเปรียบเทียบประสิทธิภาพ

In [ ]:
import time

test_prompts_perf = [
    "เมืองหลวงของฝรั่งเศสคือที่ไหน?",
    "อธิบาย machine learning ในประโยคเดียว",
    "เขียน haiku เกี่ยวกับการเขียนโค้ด"
]

print("\n" + "="*60)
print("การทดสอบความเร็ว INFERENCE")
print("="*60)

total_time = 0
for prompt in test_prompts_perf:
    start = time.time()
    _ = deepseek_pipe(prompt, max_new_tokens=50)[0]['generated_text']
    elapsed = time.time() - start
    total_time += elapsed
    print(f"Prompt: {prompt[:40]:40s} | เวลา: {elapsed:.2f}s")

avg_time = total_time / len(test_prompts_perf)
print(f"\nเวลา inference เฉลี่ย: {avg_time:.2f}s")
print(f"ขนาดโมเดล: ~1.5B พารามิเตอร์")
print(f"การใช้หน่วยความจำ: ~3-4 GB VRAM (FP16)")